In [21]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Tiingo API Key
API_KEY = 'e9099efb79717f8ff3d48cd67ab8ae71a6f71966'  # Replace with your actual API key

In [28]:
def get_trading_data(ticker, start_date, end_date):
    # Construct the API URL
    headers = {'Content-Type': 'application/json'}
    url = f"https://api.tiingo.com/iex/{ticker}/prices?startDate={start_date}&endDate={end_date}&resampleFreq=5min&afterHours=false&columns=open,high,low,close,volume&token={API_KEY}"
#     params = {
#         'token': API_KEY,
#         'startDate': start_date,
#         'endDate': end_date,
#         'resampleFreq': '5min',
#         'afterHours': 'false',
#         'columns': ['open','high','low','close','volume']
#     }

    # Make the request
    
    response = requests.get(url, headers=headers)
    data = response.json()
    return data


def filter_trading_hours(data):
    trading_data = []
    for item in data:
        # Parse and convert the timestamp to datetime object with timezone info
        timestamp = datetime.fromisoformat(item['date'].replace('Z', '+00:00')).astimezone()
        
        # Filter for active trading hours (9:30 AM to 4:00 PM New York time)
        if timestamp.hour >= 9 and timestamp.minute >= 30 and timestamp.hour < 16:
            # Remove timezone information
            item['date'] = timestamp.replace(tzinfo=None).isoformat()
            trading_data.append(item)
    return trading_data

def get_last_n_trading_days(ticker, n_days):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=10)  # Look back 10 days to ensure 3 trading days are covered

    raw_data = get_trading_data(ticker, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    print(raw_data)
    if type(raw_data) != list and raw_data['detail']:
        return None
    filtered_data = filter_trading_hours(raw_data)

    # Convert to DataFrame
    df = pd.DataFrame(filtered_data)
    df['date'] = pd.to_datetime(df['date'])

    # Ensure only the last n trading days are included
    trading_days = df['date'].dt.date.unique()[-n_days:]
    df = df[df['date'].dt.date.isin(trading_days)]

    # Remove timezone information if any remains
    df['date'] = df['date'].dt.tz_localize(None)
    df.insert(1, 'ticker', ticker)
    
    return df


In [56]:
#Get all the tickers for F500 listed in the US Stock Exchange
file_path = r"C:\Users\Devansh\Desktop\Maze-Solver\Stock-price-prediction\F500.csv"
f500 = pd.read_csv(file_path)
tickers_f500 = list(f500["Symbol"][:40])
print(tickers_f500)

['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOG', 'GOOGL', 'META', 'BRK/B', 'BRK/A', 'LLY', 'TSLA', 'AVGO', 'WMT', 'JPM', 'UNH', 'V', 'XOM', 'MA', 'ORCL', 'PG', 'JNJ', 'COST', 'HD', 'ABBV', 'KO', 'BAC', 'MRK', 'NFLX', 'ADBE', 'CVX', 'PEP', 'TMO', 'CRM', 'AMD', 'TMUS', 'LIN', 'CCZ', 'MCD', 'ABT', 'DHR']


In [29]:
tickers = [
    "AAPL"
#     , "MSFT", "AMZN", "GOOGL", "FB", "TSLA", "BRK.B", "NVDA", "JPM", "JNJ",
#     "V", "PG", "UNH", "HD", "DIS", "MA", "PYPL", "NFLX", "ADBE", "INTC",
#     "VZ", "CSCO", "PEP", "T", "KO", "MRK", "PFE", "ABT", "NKE", "ORCL",
#     "CRM", "XOM", "LLY", "WMT", "QCOM", "BAC", "ABBV", "COST", "MCD", "AMD",
    # "MDT", "HON", "AMGN", "NEE", "LOW", "BA", "AVGO", "UPS", "TXN", "UNP",
    # "CVX", "IBM", "DHR", "SBUX", "BMY", "MMM", "GS", "RTX", "BLK", "LIN",
    # "GE", "ISRG", "CAT", "SPGI", "TMO", "INTU", "PLD", "MO", "MS", "SCHW",
    # "LMT", "TJX", "AMT", "EL", "DE", "ADP", "SYK", "MDLZ", "ZTS", "C",
    # "BKNG", "NOW", "AXP", "CB", "ADI", "TGT", "MMC", "CSX", "DUK", "GILD",
    # "CCI", "ADSK", "FIS", "ICE", "MU", "SO", "REGN", "EW", "CL", "EQIX"
]
print(len(tickers))

1


In [30]:
df = pd.DataFrame()

for ticker in tickers:
    df_ticker = get_last_n_trading_days(ticker, 3)
    if df_ticker is not None:
        df = pd.concat([df, df_ticker], ignore_index=False)

print(df.head)

def save_to_excel(df, filename):
    # Save DataFrame to Excel
    df.to_excel(filename, index=False)


[{'date': '2024-09-03T13:30:00.000Z', 'open': 228.615, 'high': 229.0, 'low': 226.925, 'close': 227.015, 'volume': 26250.0}, {'date': '2024-09-03T13:35:00.000Z', 'open': 227.05, 'high': 227.685, 'low': 226.625, 'close': 227.09, 'volume': 14130.0}, {'date': '2024-09-03T13:40:00.000Z', 'open': 227.07, 'high': 227.155, 'low': 226.035, 'close': 226.035, 'volume': 21368.0}, {'date': '2024-09-03T13:45:00.000Z', 'open': 226.035, 'high': 226.14, 'low': 225.315, 'close': 226.135, 'volume': 22051.0}, {'date': '2024-09-03T13:50:00.000Z', 'open': 226.125, 'high': 226.715, 'low': 225.55, 'close': 225.65, 'volume': 9265.0}, {'date': '2024-09-03T13:55:00.000Z', 'open': 225.54, 'high': 226.24, 'low': 225.52, 'close': 226.17, 'volume': 17997.0}, {'date': '2024-09-03T14:00:00.000Z', 'open': 225.91, 'high': 226.49, 'low': 225.56, 'close': 225.94, 'volume': 9782.0}, {'date': '2024-09-03T14:05:00.000Z', 'open': 225.96, 'high': 226.27, 'low': 225.39, 'close': 225.535, 'volume': 8036.0}, {'date': '2024-09-03T

In [6]:
df.to_csv('Trading data.csv', encoding='utf-8')

In [31]:
print(df_ticker.columns)

Index(['date', 'ticker', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
